In [1]:
import pyomo.environ as pyo
import numpy as np
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

In [2]:
class LogDetModel(ExternalGreyBoxModel):
    def __init__(self, use_exact_derivatives=True,verbose=False):
        self._use_exact_derivatives = use_exact_derivatives
        self.verbose = verbose

        # For use with exact Hessian
        self._output_con_mult_values = np.zeros(1)

        if not use_exact_derivatives:
            raise NotImplementedError("use_exact_derivatives == False not supported")
        
    def input_names(self):
        return ['a', 'b']

    def equality_constraint_names(self):
        # no equality constraints
        return [ ]
    
    def output_names(self):
        return ['log_det']

    def set_output_constraint_multipliers(self, output_con_multiplier_values):
        assert len(output_con_multiplier_values) == 1
        np.copyto(self._output_con_mult_values, output_con_multiplier_values)

    def finalize_block_construction(self, pyomo_block):
        # set lower bounds on the variables
        pyomo_block.inputs['a'].setlb(-10)
        pyomo_block.inputs['b'].setlb(-1)

        # set upper bounds on the variables
        pyomo_block.inputs['a'].setub(10)
        pyomo_block.inputs['b'].setub(1)

        # initialize the variables
        pyomo_block.inputs['a'].value = 1
        pyomo_block.inputs['b'].value = 1

    def set_input_values(self, input_values):
        self._input_values = list(input_values)

    def evaluate_equality_constraints(self):

        # Not sure what this function should return with no equality constraints
        return None
    
    def evaluate_outputs(self):
        a = self._input_values[0]
        b = self._input_values[1]

        # form matrix
        M = np.array([[a,b],[b,a]])

        # compute log determinant
        (sign, logdet) = np.linalg.slogdet(M)

        if self.verbose:
            print("\n Consider M =\n",M)
            print("   logdet = ",logdet,"\n")

        return np.asarray([logdet], dtype=np.float64)

    def evaluate_jacobian_equality_constraints(self):
        return None

    def evaluate_jacobian_outputs(self):

        a = self._input_values[0]
        b = self._input_values[1]

        if self._use_exact_derivatives:

            # form matrix
            M = np.array([[a,b],[b,a]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            row = np.zeros(2)
            col = np.zeros(2)
            data = np.zeros(2)
            row[0], col[0], data[0] = (0, 0, Minv[0,0])
            row[1], col[1], data[1] = (0, 1, Minv[0,1])
            return coo_matrix((data, (row, col)), shape=(1,2))


    def evaluate_hessian_outputs(self):

        a = self._input_values[0]
        b = self._input_values[1]

        if self._use_exact_derivatives:

            # form matrix
            M = np.array([[a,b],[b,a]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            # Let f(X) = logdet(X)
            # Minv = [ [df/da, df/db],
            #          [df/db, df/da] ]

            # Want Hessian
            # [ [d2f/da2,  d2d/dadb],
            #   [d2f/dadb, d2f/db2 ] ]

            # Let S1 = dX / da, thus
            # S1 = [ [1, 0], 
            #        [0, 1] ]
            #
            # Minv @ S1 @ Minv =
            # [ [df2/da2, df2/dbda], 
            #   [df2/dbda, df2/da2]

            # Similarly
            # Let S2 = dX / db, thus
            # S2 = [ [0, 1], 
            #        [1, 0] ]
            #
            # Minv @ S2 @ Minv =
            # [ [df2/dadb, df2/db2], 
            #   [df2/db2, df2/dadb]

            S1 = np.array([[1, 0],[0,1]])
            temp1 = -Minv @ S1 @ Minv

            S2 = np.array([[0,1],[1,0]])
            temp2 = -Minv @ S2 @ Minv

            # Extract values
            df2da2 = temp1[0,0]
            df2dadb = temp1[0,1]
            df2db2 = temp2[0,1]

            lam = self._output_con_mult_values
        
            # Note: Only return low triangular of Hessian
            nnz = 3
            irow = np.zeros(nnz, dtype=np.int64)
            jcol = np.zeros(nnz, dtype=np.int64)
            data = np.zeros(nnz, dtype=np.float64)

            idx = 0
            irow[idx], jcol[idx], data[idx] = (0, 0, lam[0]*df2da2)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (1, 0, lam[0]*df2dadb)
            idx += 1
            irow[idx], jcol[idx], data[idx] = (1, 1, lam[0]*df2db2)
            idx += 1

        assert idx == nnz
        hess = coo_matrix( (data, (irow,jcol)), shape=(2,2) )
        return hess

In [3]:
def maximize_log_det(show_solver_log=True, additional_options={}):
    m = pyo.ConcreteModel()

    # create a block to store the external grey box model
    m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel())
    
    # add objective to maximize log det
    m.obj = pyo.Objective(expr=m.my_block.outputs['log_det'], sense=pyo.maximize)

    solver = pyo.SolverFactory('cyipopt')

    #solver.config.options['hessian_approximation'] = 'limited-memory'

    for k,v in additional_options.items():
        solver.config.options[k] = v
        
    results = solver.solve(m, tee=show_solver_log)
    pyo.assert_optimal_termination(results)
    return m

In [4]:
m = maximize_log_det(additional_options={'max_iter':20})

In [5]:
print("a = ",m.my_block.inputs['a'].value)

a =  10.000000074940964


In [6]:
print("b = ",m.my_block.inputs['b'].value)

b =  2.938986019274615e-16
